# Notebook de Asistente de Trámites Municipales (RAG con LangChain y Gemini)

Este notebook contiene el código para crear un sistema RAG (Retrieval-Augmented Generation) que indexa documentos PDF de trámites de Formosa y responde preguntas utilizando Gemini (Google), Ollama para embeddings y ChromaDB para la base de datos vectorial.

## 1. Instalación de Dependencias

Ejecuta esta celda una sola vez para asegurar que tienes todas las librerías necesarias.

pip install langchain langchain-google-genai pypdf chromadb langchain-ollama langchain-text-splitters python-dotenv

## 2. Configuración y Utilidades Base

Esta celda configura la clave API de Google, define la ubicación de los documentos (./tramites) y establece el modelo de embedding de Ollama.

Importante: Asegúrate de que tu archivo .env contenga la línea API_KEY="TU_CLAVE_DE_GEMINI" y que el servidor Ollama esté activo y tenga el modelo nomic-embed-text:latest descargado.

In [1]:
import os
from dotenv import load_dotenv

# Dependencias de LangChain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma 
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:

load_dotenv()
api_key = os.getenv("API_KEY")
carpeta = "./tramites" # Carpeta donde deben estar tus PDFs

if not os.path.exists(carpeta):
    os.makedirs(carpeta)
    print(f" Carpeta '{carpeta}' creada. Por favor, coloque sus PDFs dentro.")

### Funciones de Embedding y Vector Store

Usaremos Ollama como nuestro modelo de embedding local y Chroma como vector store

In [3]:
try:
    embedding = OllamaEmbeddings(
        model="nomic-embed-text:latest"
    )
    print(" Ollama Embeddings configurado.")
except Exception as e:
    print(f" ERROR: Falló la configuración de Ollama. ¿Está el servidor activo? {e}")


def get_vector_store(name_collection: str): 
    
    vector_store = Chroma(
        collection_name=name_collection,
        embedding_function=embedding,
        persist_directory="./vectorstore"
    )
    return vector_store




 Ollama Embeddings configurado.


## 3. Funciones de Carga y Segmentación (Chunking)

Definimos las funciones que extraen el texto de los PDFs y lo dividen en fragmentos lógicos.

In [4]:
def upload_pdf(path: str):
    
    try:
        loader = PyMuPDFLoader(path)
        pages = loader.load()
        # Mejor práctica: unir el contenido de las páginas directamente
        text = "\n".join([p.page_content for p in pages])
        print(f" PDF cargado correctamente ({len(pages)} páginas, {len(text)} caracteres)")
        return text
    except Exception as e:
        # Esto captura errores si el PDF está corrupto o protegido
        print(f" Error al cargar PDF {path}: {e}")
        return ""

def split_text(text: str): 
    
    # RecursiveCharacterTextSplitter respeta la estructura de documentos (saltos de línea)
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=3000,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " "] # Separadores jerárquicos
    )
    docs = splitter.create_documents([text])
    print(f" {len(docs)} fragmentos creados.")
    return docs


def retrieval(input_user: str): 
    
    vector_store = get_vector_store("tramites_formosa")
    # Aumentar k a 10 para obtener más contexto
    docs = vector_store.similarity_search(input_user, k=10) 
    return docs

## 4. Fase de Indexación: Leer, Dividir y Guardar

Este es el bloque procesa todos los archivos en la carpeta ./tramites.

In [5]:
print("\n--- INICIANDO FASE DE INDEXACIÓN ---")
vector_store = get_vector_store("tramites_formosa")
archivos_indexados = 0
for archivo in os.listdir(carpeta):
    if archivo.endswith(".pdf"):
        ruta = os.path.join(carpeta, archivo)
        print(f"\n Leyendo: {archivo}")

        # 1. Cargar el PDF
        text = upload_pdf(ruta) 

        if text.strip():
            
            # 2. Dividir el texto en fragmentos (chunks)
            docs = split_text(text) 
            
            if docs:
                try:
                    # 3. Guardar en ChromaDB

                    # Esta línea genera los embeddings con Ollama y los guarda en el disco
                    vector_store.add_documents(docs) 
                    print(f" Fragmentos de {archivo} guardados en ChromaDB.")
                    archivos_indexados += 1
                except Exception as e:
                     # Captura fallos de Ollama o ChromaDB
                     print(f" Error al guardar embeddings de {archivo}: {e}")
            else:
                print(f" El archivo {archivo} se cargó, pero no generó fragmentos.")
        else:
            print(f" El archivo {archivo} no contiene texto útil.")

print(f"\n--- INDEXACIÓN COMPLETADA: {archivos_indexados} archivos procesados ---")


--- INICIANDO FASE DE INDEXACIÓN ---

 Leyendo: ASPECTOS LEGALES DE LA DOCUMENTACIÓN A PRESENTAR.pdf
 PDF cargado correctamente (1 páginas, 3266 caracteres)
 2 fragmentos creados.
 Fragmentos de ASPECTOS LEGALES DE LA DOCUMENTACIÓN A PRESENTAR.pdf guardados en ChromaDB.

 Leyendo: DOCUMENTACIÓN A PRESENTAR PARA OBRA NUEVA.pdf
 PDF cargado correctamente (3 páginas, 2611 caracteres)
 1 fragmentos creados.
 Fragmentos de DOCUMENTACIÓN A PRESENTAR PARA OBRA NUEVA.pdf guardados en ChromaDB.

 Leyendo: Estacionamiento-Medido.pdf
 PDF cargado correctamente (3 páginas, 2475 caracteres)
 1 fragmentos creados.
 Fragmentos de Estacionamiento-Medido.pdf guardados en ChromaDB.

 Leyendo: Recolección de residuos.pdf
 PDF cargado correctamente (10 páginas, 9873 caracteres)
 4 fragmentos creados.
 Fragmentos de Recolección de residuos.pdf guardados en ChromaDB.

 Leyendo: Requisitos para afectar remises.pdf
 PDF cargado correctamente (4 páginas, 4116 caracteres)
 2 fragmentos creados.
 Fragmentos de 

## 5. Función de Respuesta (Generación con Gemini)

Esta celda configura el prompt y la conexión con el modelo Gemini para generar la respuesta final.

In [6]:

prompt = PromptTemplate.from_template("""
    Eres un asistente encargado de responder preguntas sobre tramites de la municipalidad de Formosa.
    Usa exclusivamente la informacion del contexto para responder al usuario.
    contexto = {contexto}
    pregunta del usuario: {input_user}
                                    
    Responde de forma clara, concisa y completa, incluyendo todos los requisitos relevantes y mencionando sedes o direcciones si aplica.
""")

def response(input_user: str, contexto: str):
    llm = ChatGoogleGenerativeAI(
        api_key=api_key,
        model="gemini-2.5-flash", 
        temperature=0.4
    )
    # Stream para mejor experiencia de usuario
    print("\n--- Respuesta del Asistente ---")
    for chunk in llm.stream(prompt.format(contexto=contexto, input_user=input_user)):
        yield chunk.content
    print("\n-------------------------------\n")

## 6. Consulta y Generación (Fase Final del RAG)

Ejecuta esta celda para interactuar con tu base de conocimiento.

In [ ]:
input_user =input("Human: ")

# 1. Recuperación (Retrieval)
docs = retrieval(input_user=input_user)

if docs and docs[0].page_content.strip(): # Verificar que se encontró contenido
    
    contexto = "\n".join([doc.page_content for doc in docs])
    
    # 2. Generación (Generation)
    for chunk in response(input_user=input_user, contexto=contexto):
        print(chunk, end="", flush=True)

else:
    print("\n No se encontraron documentos relevantes en la base de datos para responder a la pregunta.")